# Политика сохранения
I. Создаём резервную копию на каждое сохранение файла excel
    - Служба запускается каждые 30 минут.
    1. Проверяем: (время последнего изменения файла)
    file_state != last_state
        1.1 Состояние файла хранится в служебном файле .dump.state

    2. Если состояние изменилось - делаем дамп
        <!-- 2.1 Сохраняем дамп ячеек только по сегодняшней дате -->

    3. Удаляем самый старый дамп, если их больше 30

II. На каждое (значимое) изменение файла в процессе работы 
<!-- ?? Возможно ли отслеживать изменения во время работы? -->
    -? Если существует файл "~/celWorkbook" - сохранять состояние на каждое изменение

In [15]:
source_file = './files/source.xlsx'
dump_dir  = './files/dump/'
dump_state_file = './files/dump/.dump.state'

In [5]:
import openpyxl
import csv
import os
import sys

from datetime import datetime

In [18]:
# Сохранить дамп
def dump_save(source: str, dump_dir: str):
    DUMP_FILENAME = 'dump'
    
    wb = openpyxl.load_workbook(source)
    sh = wb.active  #wb['Sheet1']

    
    savetime = datetime.now().strftime('%d.%m.%Y - %H.%M')
    dump = dump_dir + DUMP_FILENAME + '_' + savetime + '.csv'

    with open(dump, 'w', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        for row in sh.rows:
            f_writer.writerow([cell.value for cell in row])

In [35]:
def dump_cleanup(dump_dir: str):
    file_list = [file for file in os.listdir(dump_dir)]
    file_list.remove('.dump.state')

    if len(file_list) > 30:
        file_list.sort()
        os.remove(dump_dir + file_list[0])

In [36]:
# Обновить .dump.state
def dump_update():
    # file_state получили 
    file_state = datetime.fromtimestamp(
                    os.path.getmtime(source_file)
                ).strftime('%Y.%m.%d - %H.%M')


    if not os.path.exists(dump_dir):
        os.mkdir(dump_dir)
    else:
        # сравнили
        with open(dump_state_file, 'r', encoding='utf-8') as f:
            last_state = f.read()

    # если не равны
    if file_state != last_state or last_state is None:
        # 1.сохранили dump
        dump_save(source_file, dump_dir)

        # 2.записали новый last_state
        with open(dump_state_file, 'w', encoding='utf-8') as f:
            f.write(file_state)
    
    # исполнить dump policy (очистить дамп)
    dump_cleanup(dump_dir)


In [5]:
# TESTING
try:
    source = './file/source.xlsx'
    wb = openpyxl.load_workbook(source)
except FileNotFoundError:
    print("FileNotFoundError")

FileNotFoundError
